In [5]:
from langgraph.graph import StateGraph, END,START
from langchain_openai import ChatOpenAI
from typing import TypedDict,Literal
from dotenv import load_dotenv
from pydantic import BaseModel , Field


In [7]:
load_dotenv()
model=ChatOpenAI(model="gpt-4o-mini")

In [25]:
class SentimentSchema(BaseModel):
    sentiment:Literal["positive","negative"]= Field(description="sentiment of the review")

class diagnosisSchema(BaseModel):
    issue_type:Literal["UX","performance","Bug","Support","other"]=Field(description='the issue type mention in the review')
    tone:Literal["angry","calm","frustrated","other"]=Field(description="the emotion and tone of the review")
    urgency:Literal["high","medium","low"]=Field(description="the urgency of the issue like how urgent and critical the issue is")

In [26]:
structure_model=model.with_structured_output(SentimentSchema)
diagnosis_model=model.with_structured_output(diagnosisSchema)



In [11]:
class ReviewState(TypedDict):
    review:str
    sentiment:Literal["positive","negative"]
    diagnosis:dict
    response:str

In [10]:
prompt='what is the sentiment of the following review.'

In [32]:
def find_sentiment(state:ReviewState):
    prompt=f'For the following review find out the sentiment {state["review"]}'
    sentiment=structure_model.invoke(prompt).sentiment
    return{"sentiment":sentiment}

def check_sentiment(state:ReviewState)->Literal["positive_resonse","run_diagnosis"]:
    if state["sentiment"]== "positive":
        return "positive_response"
    else:
        return "run_diagnosis"

def positive_resonse(state:ReviewState):
    prompt=f'write a short and sweet response to the following review {state["review"]}'
    response=model.invoke(prompt).content
    return{"response":response}

def run_diagnosis(state:ReviewState):
    prompt=f'Diagnosis the following review : {state["review"]} \n return issue type , tone and urgency'
    response = diagnosis_model.invoke(prompt)
    return {"diagnosis":response.model_dump()}

def negative_resonse(state:ReviewState):
    diagnosis=state["diagnosis"]
    prompt=f'you are the support assistant. User had a {diagnosis["issue_type"]} issue with {diagnosis["tone"]} tone and {diagnosis["urgency"]} urgency. \n write a short and sweet response to the following review {state["review"]}'
    response=model.invoke(prompt).content
    return{"response":response}



In [34]:
graph=StateGraph(ReviewState)
graph.add_node("find_sentiment",find_sentiment)
graph.add_node("positive_resonse",positive_resonse)
graph.add_node("run_diagnosis",run_diagnosis)
graph.add_node("negative_resonse",negative_resonse)

graph.add_edge(START,"find_sentiment")
graph.add_conditional_edges("find_sentiment",check_sentiment)
graph.add_edge("positive_resonse", END)
graph.add_edge("run_diagnosis","negative_resonse")
graph.add_edge("negative_resonse", END)
workflow=graph.compile()


In [37]:
initial_state={"review":"the app is good but it crashes sometimes"}
workflow.invoke(initial_state)


{'review': 'the app is good but it crashes sometimes',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'calm', 'urgency': 'medium'},
 'response': "Thank you for your feedback! We're glad to hear you enjoy the app, but we’re sorry to hear about the crashes. Our team is working diligently to resolve this issue. Please make sure you’re using the latest version, and feel free to share any specific details that could help us improve. Your support means a lot to us!"}